In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
"""Labeling Data.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1vwQneJX92WYgVynrmhDm9RINVzKDtSwa
"""

# Install required packages
!pip install pandas

from google.colab import drive
drive.mount('/content/drive')

import os
import openai
import pandas as pd
import re

# Load the data from Excel
file_path = '/content/drive/My Drive/Additional Education/NJIT/Classes/DS677-850 Deep Learning/Project/Work Data/synthetic_data.xlsx'
data = pd.read_excel(file_path)

Mounted at /content/drive


In [ ]:
openai.api_key = 'api_key'

In [ ]:
# Function to generate question-answer pairs
def generate_qa_pairs(long_text):
    prompt = (
        f"Given the following incident description:\n{long_text}\n"
        "Generate a relevant question and provide an answer based on the context. "
        "Ensure the question is clear, specific, and directly related to the incident. "
        "The answer must be a verbatim extract from the provided context, not a summary. "
        "The questions and answers will be used to train a simple BERT model."
        "Create only a single question / answer pair."
        "Keep the questions fairly generic, maybe referring to equipment categories."
        "An example question is: Why is circ. pump motor stator temp high?"
        "Format the output as:\nQuestion: <your question>\nAnswer: <your answer>"
    )

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert question-answer generator."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )

    qa_pair = response.choices[0].message.content.strip()
    return qa_pair


In [ ]:
print(data["LongText"][0])

* **************Incident Description************** * Notification 000030761040 Details * *********************************************************** * 03/15/2019 15:26:32 INCIDENT REPORT (INCIDENT REPORT) * Report submitted by: James Franklin ENTITY\INCIDENT: * * CONDITION DESCRIPTION/LOCATION (THE INAPPROPRIATE ACTION AND ITS NEGATIVE EFFECT/INCLUDE A DETAILED LOCATION DESCRIPTION): * During regular inspection on 03/15/2019, 18B Pump was found operating at a significantly lower pressure than usual. Initial measurement showed 34 psig, which is lower than the average operating pressure of the pump. All necessary pressure checks were made  and potential leak points inspected. Noise and heat generation appear to be in normal range, but the decreased pressure could indicate an internal compromise in the pump or a problem in the supply line. * * ACTIVITIES, PROCESSES, PROCEDURES INVOLVED: * Regular monitoring and safety checks  * * WHY DID CONDITION HAPPEN? * Cause currently unknown, suspec

In [ ]:
generate_qa_pairs(data["LongText"][0])

'Question: What were the immediate actions recommended after the discovery of abnormally low pressure in the 18B Pump?\nAnswer: Isolate 18B Pump and investigate for internal damage or supply line obstructions. Repair or replacement may be required, pending investigation results.'

In [ ]:
# Generate questions and answers for each synthetic entry
qa_data = []
for index, row in data.iterrows():
    long_text = row['LongText']
    qa_pair = generate_qa_pairs(long_text)

    # Extract question and answer from the generated pair
    question_match = re.search(r'Question: (.+?)\n', qa_pair)
    answer_match = re.search(r'Answer: (.+)', qa_pair)

    if question_match and answer_match:
        question = question_match.group(1).strip()
        answer = answer_match.group(1).strip()

        qa_data.append({
            'CreatedOn': row['CreatedOn'],
            'Notification': row['Notification'],
            'OrderNum': row['OrderNum'],
            'ShortText': row['ShortText'],
            'FLOC': row['FLOC'],
            'LongText': row['LongText'],
            'Question': question,
            'Answer': answer
        })

In [ ]:
qa_data[0]

{'CreatedOn': '2019-03-15',
 'Notification': 30761040,
 'OrderNum': 90231415,
 'ShortText': 'Abrupt decrease in 18B Pump pressure',
 'FLOC': 'S4CW -4CWE7',
 'LongText': '* **************Incident Description************** * Notification 000030761040 Details * *********************************************************** * 03/15/2019 15:26:32 INCIDENT REPORT (INCIDENT REPORT) * Report submitted by: James Franklin ENTITY\\INCIDENT: * * CONDITION DESCRIPTION/LOCATION (THE INAPPROPRIATE ACTION AND ITS NEGATIVE EFFECT/INCLUDE A DETAILED LOCATION DESCRIPTION): * During regular inspection on 03/15/2019, 18B Pump was found operating at a significantly lower pressure than usual. Initial measurement showed 34 psig, which is lower than the average operating pressure of the pump. All necessary pressure checks were made  and potential leak points inspected. Noise and heat generation appear to be in normal range, but the decreased pressure could indicate an internal compromise in the pump or a problem 

In [ ]:
qa_df = pd.DataFrame(qa_data)

In [ ]:
qa_df

,CreatedOn,Notification,OrderNum,ShortText,FLOC,LongText,Question,Answer
0,2019-03-15,30761040,90231415,Abrupt decrease in 18B Pump pressure,S4CW -4CWE7,* **************Incident Description**********...,What could likely be the cause of the abnormal...,"Based on the preliminary investigation, it is ..."
1,2020-11-18,30958076,80230196,Reduced output from 5B Generator,S5GN -5GNE2-GENR,* **************Incident Description**********...,What was the issue observed in the 5B Generato...,The 5B Generator output was reduced by 20% aft...
2,2018-09-15,30751256,80212345,Rapid frequency change in Pump 18A,S4CW -4CWE5,* **************Incident Description**********...,What was the cause of the frequency fluctuatio...,The initial assessments suggest that the frequ...
3,2019-11-17,30958230,80523058,Unusual noise from 3B Pump motor,S5CW -5CWE2,* **************Incident Description**********...,What were the immediate and recommended action...,The immediate and recommended actions after th...
4,2016-09-15,30751904,80225619,Fuse M87 tripped,S4EL -4EL1746,* **************Incident Description**********...,What was the consequence of the issue with fus...,The consequence of the issue with fuse M87 was...
...,...,...,...,...,...,...,...,...
474,2018-12-27,40732176,90234712,Overheating in Converter G4,S4AC -4ACE7-TG4,* **************Incident Description**********...,What immediate actions were taken after the ov...,The immediate actions that were taken included...
475,2020-12-12,30991234,80298250,18A Motor Overheating observed,S4CW -4CWE2,* **************Incident Description**********...,What immediate actions and recommended actions...,The immediate inspection and necessary mainten...
476,2019-04-30,30987450,80239876,Unusual vibration in 18B Blower,S5BL -3BLE6,* **************Incident Description**********...,What was the anomaly reported in the blower un...,A higher than usual vibration was detected fro...
477,2021-09-15,30985240,80589752,Abnormal vibration in Fan 5A,S5AI -5AIE3,* **************Incident Description**********...,What was the cause of the abnormal vibration i...,The abnormal vibration in Fan 5A was caused by...


In [ ]:
qa_df.to_excel("/content/drive/My Drive/Additional Education/NJIT/Classes/DS677-850 Deep Learning/Project/Work Data/labeled_synthetic_data (specific questions and answers).xlsx")